In [ ]:
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from gensim.models import Word2Vec
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
!pip install utils
from utils import *
import random
import warnings
warnings.filterwarnings('ignore') 
#from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import nltk
nltk.download('punkt')
from spacy.lang.tr import Turkish
from tqdm import tqdm
import string
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

In [ ]:
column = ['tweets','duygu']#,'preprocessing'
df = pd.read_excel("../dataset/kemik_total.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    
    return accuracy_score(y_true, y_pred) # return accuracy score

In [ ]:
def classification_report_with_precision_score(y_true, y_pred):
    
    return precision_score(y_true, y_pred, average='weighted')

In [ ]:
def classification_report_with_recall_score(y_true, y_pred):
    
    return recall_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_f1_score(y_true, y_pred):
    
    return f1_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_mcc_score(y_true, y_pred):
    
    return matthews_corrcoef(y_true, y_pred)

In [ ]:
def classification_report_with_c_kappa(y_true, y_pred):

    return cohen_kappa_score(y_true, y_pred)

In [ ]:
X_t = df['tweets'].tolist()

In [ ]:
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))

In [ ]:
n_features=128
window_size=5
min_count=10
epoch=50
n_workers=8

word2vec = Word2Vec(size = n_features,
            window = window_size, 
            min_count= min_count,
            workers = n_workers, 
            sg=1)
word2vec.build_vocab(X_t)
word2vec.train(X_t, 
            total_examples=word2vec.corpus_count,  
            epochs = epoch)

In [ ]:
nlp= Turkish()
def feature_extraction_word2vec(text):
    yeniMetin = ""
    for i in text:
        if i not in string.punctuation:
            yeniMetin += i
    vector=np.zeros(32) #400 uzunluğunda boş sıfırlarla bir vektör oluşturdum
    for token in nlp(yeniMetin):
        word = token.text.lower()
        if word not in ['',' ']:
            if word in word2vec.wv.vocab: #word2vec.wv.key_to_index.keys()
                vector+= word2vec.wv[word] #gensim modeliyle oluşturulan modeli kullandım
                #print(vector)
        
    return vector

In [ ]:
y_total= []
y_total=df.duygu.tolist()

X_total = []
for element in tqdm(df.tweets):
    X_total.append(feature_extraction_word2vec(element))
    

In [ ]:
# Nested CV with parameter optimization
svc = svm.SVC(kernel='linear')
clf = OneVsRestClassifier(svc)

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/SVM: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/SVM: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/SVM: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vec/SVM: ",f1.mean())

mcc_score = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_mcc_score))

print("MCC_score for word2vec/SVM: ",mcc_score.mean())

c_kappa = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_c_kappa))

print("C_kappa for word2vec/SVM: ",c_kappa.mean())

In [ ]:
clf =  LogisticRegression(multi_class='ovr')

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/LogisticRegression: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/LogisticRegression: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/LogisticRegression: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vec/LogisticRegression: ",f1.mean())

mcc_score = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_mcc_score))

print("MCC_score for word2vec/LogisticRegression: ",mcc_score.mean())

c_kappa = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_c_kappa))

print("C_kappa for word2vec/LogisticRegression: ",c_kappa.mean())

In [ ]:
"""clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10, 3), random_state=1)

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/MLPC: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/MLPC: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/MLPC: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vecF/MLPC: ",f1.mean())
"""

In [ ]:
clf = GaussianNB()

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for word2vec/NB: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for word2vec/NB: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for word2vec/NB: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for word2vec/NB: ",f1.mean())

mcc_score = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_mcc_score))

print("MCC_score for word2vec/NB: ",mcc_score.mean())

c_kappa = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_c_kappa))

print("C_kappa for word2vec/NB: ",c_kappa.mean())